In [1]:
## Полезная библиотека для решения
# %pip install HeapDict
import heapdict

# Задание на программирование [5 баллов]

**Постановка**

В этой задаче вам предстоит реализовать алгоритм поиска на графе 𝐴*, для нахождения оптимальной последовательности ходов в игре пятнашки (3 × 3). Например, пусть дано следующее начальное расположение:

|   |   |   |
|---|---|---|
| 1 | 2 | 3 |
| 4 |   | 6 |
| 7 | 5 | 8 |

Ваша программа должна найти самую короткую последовательность ходов, чтобы перевести игру в целевое расположение:

|   |   |   |
|---|---|---|
| 1 | 2 | 3 |
| 4 | 5 | 6 |
| 7 | 8 |   |

Выход вашей программы должен выглядеть примерно так: ["down", "right"], то есть для решения этой головоломки нужно сначала сдвинуть пустую клетку внизу (поменять ее местами с костяшкой снизу), а затем сдвинуть ее вправо (поменять ее местами с костяшкой справа).

**Техническию детали**

*   Текущее состояние игры будет представлено в виде Python tuple, где пустая клетка обозачается нулем. Пример выше будет выглядеть следующим образом (1, 2, 3, 4, 0, 6, 7, 5, 8).
*   Не каждая начальная позиция может быть переведена в целевое состояние. Чтобы протестировать ваше решение, может быть полезно написать функцию, которая генерирует валидные изначальные расположения. Для этого можно начать из целевой позиции и многогратно переместить пустую клетку в случайном допустимом направлении.

## Часть 1. Реализация эвристик [2 балла]

Вам нужно реализовать две эвристики в предоставленных шаблонах функций `heuristic_misplaced` и `heuristic_manhattan`.

**Задача 1. Количество неправильно расположенных костяшек [1 балла]**

Реализуйте эвристику, которая возвращает количество костяшек, расположенных не на своих местах. Пустое пространство не является костяшкой и не должно учитываться при подсчете неправильно расположенных костяшек.

In [2]:
def heuristic_misplaced(state):
    """
    Количество неправильно расположенных костяшек.
    Пустое пространство не является костяшкой и не должно
    не учитывается при подсчете неправильно размещенных костяшек.

    :param state: Tuple, представляющий текущее состояние игры.
    :return: Количество неправильно расположенных костяшек.
    """
    n = len(state) # number of all cells in the game (if 3*3 =>  len(state) = 9
    n_var = range(1, n ) # all numbers in play (1 cell is unoccupied)
    goal_state = tuple(n_var) + (0,)  # into turple, but also add zero - for that unoccupied cell; 
    # note that these are in the appropriate order: 1,..n, 0
    return sum(1 for i in range(n) if state[i] != goal_state[i] and state[i] != 0)
    # check for correct positions, ignore zero: state[i] != goal_state[i] and state[i] != 0
    # use sum to get the total number of misplaced values - get 1 in case of the checks are completed

In [3]:
# В этом блоке вы можете протестировать свое решение для самопроверки.
# После сдачи мы протестируем ваше решение на дополнительных тестах и проверим
# корректность вручную

def test_heuristic_misplaced():
    res = []
    test_cases = [
    ((1, 2, 3, 4, 0, 6, 7, 5, 8), 2),
    ((1, 2, 3, 4, 5, 6, 7, 8, 0), 0),
    ((1, 2, 3, 4, 5, 6, 7, 0, 8), 1),
    ((1, 2, 3, 4, 5, 6, 0, 7, 8), 2),
    ((1, 2, 3, 4, 0, 5, 7, 8, 6), 2),
]



    for i, (state, expected_value) in enumerate(test_cases):
        actual_value = heuristic_misplaced(state)
        assert actual_value == expected_value, f"Тест для расположения {state} завершился ошибкой: " \
                                               f"функия heuristic_misplaced вернула значение {actual_value}, " \
                                               f"ожидаемое значение: {expected_value}"

    print("Все тесты для heuristic_misplaced завершились успешно!")

test_heuristic_misplaced()

Все тесты для heuristic_misplaced завершились успешно!


Great!

**Задача 2. Сумма манхэттенских расстояний до правильных позиций [1 балла]**

Реализуйте эвристику, которая для каждой костяшки подсчитывает манхэттэнское расстояние до правильной позиции и возвращает сумму этих расстояний. Пустое пространство не является костяшкой и не должно учитываться при подсчете.

*Пример: В приведенном выше примере расстояния от неправильно расположенных костяшек 5 и 8 до их правильных позиций равны 1, поэтому сумма манхэттенских расстояний равна 2.*

In [4]:
def heuristic_manhattan(state):
    """
    Сумма манхэттенских расстояний от неправильно
    расположенных костяшек до их правильного положения.

    :param state: Tuple, представляющий текущее состояние игры.
    :return: Сумма манхэттенских расстояний.
    """

    m = int(len(state) ** 0.5)  # puzzle size (basically just calculate the dimension of the square side))
    total_distance = 0 # to keep track of the total Manhattan distances from current to correct positions
    
    for i in range(len(state)): #iterate through all cells (tiles)
        value = state[i] # get the value of the tile
        if value != 0:  # ignore the blank tile
            goal_row, goal_col = divmod(value - 1, m)  # expected position
            current_row, current_col = divmod(i, m)    # current position
            ''' 
            divmod:
            
            quotient, remainder = divmod(a, b)
            quotient = a // b  # Integer division: 10 // 3 = 3 → (quotient) - this is the row number
            remainder = a % b  # Modulus: 10 % 3 = 1 → (remainder) - this is the column number
            
            '''
            total_distance += abs(goal_row - current_row) + abs(goal_col - current_col) # Manhattan distance
    return total_distance

    return -1 # TODO: Замените на ваше значение.

In [5]:
# В этом блоке вы можете протестировать свое решение для самопроверки.
# После сдачи мы протестируем ваше решение на дополнительных тестах и проверим
# корректность вручную

def test_heuristic_manhattan():
    res = []
    test_cases = [
        ((1, 2, 3, 4, 0, 6, 7, 5, 8), 2),
        ((1, 2, 3, 4, 5, 6, 7, 8, 0), 0),
        ((1, 2, 3, 4, 5, 6, 0, 7, 8), 2),
        ((1, 2, 3, 4, 6, 0, 7, 5, 8), 3),
        ((1, 3, 0, 4, 2, 5, 7, 8, 6), 4)
    ]

    for state, expected_value in test_cases:
        actual_value = heuristic_manhattan(state)
        assert actual_value == expected_value, f"Тест для расположения {state} завершился ошибкой: " \
                                               f"функия heuristic_manhattan вернула значение {actual_value}, " \
                                               f"ожидаемое значение: {expected_value}"

    print("Все тесты для heuristic_manhattan завершились успешно!")

test_heuristic_manhattan()

Все тесты для heuristic_manhattan завершились успешно!


nice

## Часть 2. Реализация A* [2 балла]

Теперь вам предстоит реализовать алгоритм поиска на графе A* в предоставленном шаблоне функции astar, чтобы найти решение пятнашек с помощью эвристик, описанных выше. Ваша функция должна возвращать два объекта:

*   строку, представляющую ходы, необходимые для достижения цели
*   список состояний в том порядке, в котором они были исследованы алгоритмом (expanded)

Используйте символы «r», «l», «u» и «d» для обозначения направлений «вправо», «влево», «вверх» и «вниз» соответственно. В приведенном выше примере ваша функция должна вернуть строку `dr`. Если изначальное расположение не имеет решений, верните None для оптимального пути и список посещенных (Expanded) состояний.

**Детерминированность**

Строго говоря, A* не является детерминированым алгоритмом. Если при выборе следующей вершины для исследования несколько вершин имеют одинаковое значение cost-функции (текущее количество ходов + значение эвристики), то алгоритм выбирает из этих вершин произвольно.

Чтобы мы могли проверить ваше решение, мы требуем, чтобы ваша реализация алгоритма выбирала из нескольких вершин с одинаковой cost-функцией в лексикографическом порядке. Например, состояние (1, 2, 3, 4, 0, 6, 7, 5, 8) лексикографически меньше, чем (1, 2, 3, 4, 0, 6, 7, 8, 5).

**Реализация**

В реализации алгоритма вам потребуется очередь с приоритетом для выбора следующей вершины для исследования. Мы рекомендуем использовать heapdict. В первом блоке этого ноутбука показано как установить эту библиотеку в Google Colab и импортировать ее. Подробности можно найти по адресу https://github.com/DanielStutzbach/heapdict.

Чтобы выполнить требование детерминированности, в качестве value в очереди с приоритетами следует использовать tuple (зачение cost-функции, state). Тогда выбор из нескольких вершин с одинаковыми значениями cost-функции будет осуществляться в лексикографическом порядке.

**Тестирование**

В блоке ниже реализован простой тест для первичной проверки алгоритма. Однако, мы рекомендуем вам протестировать решение на дополнительных примерах, которые вы должны сгенерировать самостоятельно.

In [7]:
from heapdict import heapdict

def get_neighbors(state, n):
    """returns valid moves and new states"""
    state = list(state)
    zero_pos = state.index(0)  # find blank tile (0)
    row, col = divmod(zero_pos, n)

    moves = []
    move_dict = {'r': (0, 1), 'l': (0, -1), 'd': (1, 0), 'u': (-1, 0)}  # movement directions

    for move, (dr, dc) in move_dict.items():
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < n and 0 <= new_col < n:  # check valid move
            new_pos = new_row * n + new_col
            new_state = state[:]
            new_state[zero_pos], new_state[new_pos] = new_state[new_pos], new_state[zero_pos]  # swap tiles
            moves.append((move, tuple(new_state)))  # store new state
    
    return moves

def is_solvable(state):
    """checks if puzzle is solvable"""
    inv_count = sum(
        1 for i in range(len(state)) for j in range(i + 1, len(state))
        if state[i] and state[j] and state[i] > state[j]  # count inversions
    )
    n = int(len(state) ** 0.5)
    if n % 2 == 1:  # odd-sized board
        return inv_count % 2 == 0
    else:  # even-sized board
        zero_row = state.index(0) // n  # find blank tile row
        return (inv_count + zero_row) % 2 == 0  # check parity condition

def astar(init_state, heuristic):
    """
    a* search algorithm for solving n×n sliding puzzle with lexicographic ordering
    
    :param init_state: initial puzzle state (tuple)
    :param heuristic: heuristic function (h1 or h2)
    :return: (optimal path as string, list of explored states)
    """

    if not is_solvable(init_state):
        return None, []  # return if puzzle is unsolvable

    n = int(len(init_state) ** 0.5)
    goal_state = tuple(range(1, len(init_state))) + (0,)  # define goal state

    # priority queue using heapdict (f, g, path, state) → ensures lexicographic order
    open_set = heapdict()
    open_set[init_state] = (heuristic(init_state), 0, "", init_state)  # (f, g, path, state)

    came_from = {}  # track path history
    visited = set()  # track visited states
    expanded = []  # track order of explored states

    while open_set:
        current_state, (f, g, path, _) = open_set.popitem()  # pop lexicographically smallest state
        expanded.append(current_state)

        if current_state == goal_state:
            return path, expanded  # return solution if goal reached

        visited.add(current_state)

        for move, new_state in get_neighbors(current_state, n):
            if new_state in visited:
                continue  # skip already visited states

            new_g = g + 1  # update step cost
            new_f = new_g + heuristic(new_state)  # compute total cost

            # enforce lexicographic ordering by storing state in priority queue
            if new_state not in open_set or new_f < open_set[new_state][0]:
                open_set[new_state] = (new_f, new_g, path + move, new_state)  # ensure lexicographic order
                came_from[new_state] = (current_state, move)  # store path info

    return None, expanded  # return if no solution found


In [8]:
# В этом блоке вы можете протестировать свое решение для самопроверки.
# После сдачи мы протестируем ваше решение на дополнительных тестах и проверим
# корректность вручную

def test_astar(heuristic):
    test_cases = [
            ((1, 2, 3, 4, 5, 6, 7, 8, 0), "", [(1, 2, 3, 4, 5, 6, 7, 8, 0)]),
            ((1, 2, 3, 4, 5, 6, 0, 7, 8), "rr", [(1, 2, 3, 4, 5, 6, 0, 7, 8),
                                                (1, 2, 3, 4, 5, 6, 7, 0, 8),
                                                (1, 2, 3, 4, 5, 6, 7, 8, 0)]),
            ((1, 2, 3, 4, 6, 0, 7, 5, 8), "ldr", [(1, 2, 3, 4, 6, 0, 7, 5, 8),
                                                (1, 2, 3, 4, 0, 6, 7, 5, 8),
                                                (1, 2, 3, 4, 5, 6, 7, 0, 8),
                                                (1, 2, 3, 4, 5, 6, 7, 8, 0)]),
            ((1, 3, 0, 4, 2, 5, 7, 8, 6), "ldrd", [(1, 3, 0, 4, 2, 5, 7, 8, 6),
                                                (1, 0, 3, 4, 2, 5, 7, 8, 6),
                                                (1, 2, 3, 4, 0, 5, 7, 8, 6),
                                                (1, 2, 3, 4, 5, 0, 7, 8, 6),
                                                (1, 2, 3, 4, 5, 6, 7, 8, 0)]),
        ]

    for test in test_cases:
        state, actual_path, actual_states = test[0], test[1], test[2]
        path, states = astar(test[0], heuristic)
        assert path == actual_path and states == actual_states, f"Тест для расположения {state} завершился ошибкой: " \
                                                                f"astar вернула {path}, {states}, " \
                                                                    f"ожидаемые значения {actual_path, actual_states}"
    print("Все тесты для test_astar завершились успешно!")

test_astar(heuristic_misplaced)

Все тесты для test_astar завершились успешно!


## Часть 3. Сравнение эвристик [1 балл]

Теперь сравниие эффективность различных эвристик. Выберите несколько допустимых начальных позиций и сравните количество исследованных (expanded) алгоритмом состояний для двух эвристик. Прокомментируйте наблюдаемый результат.

In [9]:
def check_heuristics_difference():
    """compares the number of expanded states for different heuristics"""

    test_cases = [
        (1, 2, 3, 4, 0, 6, 7, 5, 8),  # simple swap needed
        (3, 1, 2, 6, 4, 5, 0, 7, 8),
        (8, 6, 7, 2, 5, 4, 3, 0, 1), # just some more cases))
        (6, 4, 7, 8, 5, 0, 3, 2, 1),
        (4, 1, 3, 7, 2, 5, 0, 8, 6),  
        (1, 2, 3, 4, 5, 6, 8, 7, 0),
    ]

    results = []

    for state in test_cases:
        # run A* search with both heuristics
        path_h1, expanded_h1 = astar(state, heuristic_misplaced)
        path_h2, expanded_h2 = astar(state, heuristic_manhattan)

        # check if it is an unsolvable case
        if path_h1 is None and path_h2 is None:  
            results.append(("Unsolvable", "Unsolvable"))
        else: # if not- continue
            results.append((len(expanded_h1), len(expanded_h2)))

    # print results
    for i, (h1_exp, h2_exp) in enumerate(results):
        print(f"test case {i+1}:")
        if h1_exp == "Unsolvable":
            print("  this puzzle is unsolvable.")
        else:
            print(f"  expanded states (h1 - misplaced tiles): {h1_exp}")
            print(f"  expanded states (h2 - Manhattan distance): {h2_exp}")
            print(f"  h2 is {round(h1_exp / h2_exp, 1)} times more efficient\n")

check_heuristics_difference()

test case 1:
  expanded states (h1 - misplaced tiles): 3
  expanded states (h2 - Manhattan distance): 3
  h2 is 1.0 times more efficient

test case 2:
  expanded states (h1 - misplaced tiles): 8374
  expanded states (h2 - Manhattan distance): 1428
  h2 is 5.9 times more efficient

test case 3:
  expanded states (h1 - misplaced tiles): 143849
  expanded states (h2 - Manhattan distance): 21198
  h2 is 6.8 times more efficient

test case 4:
  expanded states (h1 - misplaced tiles): 143849
  expanded states (h2 - Manhattan distance): 21198
  h2 is 6.8 times more efficient

test case 5:
  expanded states (h1 - misplaced tiles): 7
  expanded states (h2 - Manhattan distance): 7
  h2 is 1.0 times more efficient

test case 6:
  this puzzle is unsolvable.


Well, for easy cases both heuristics perform the same way. For other cases- sometimes one is better than the other or inverse. 